In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
# import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load premotor neuron table

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')
print(pre_to_mn_df.shape)
pre_to_mn_df.loc['sensory'].head()

# Create a premotor connectome

## First get all synapses by querying chunks of segids

In [ ]:
# get all the synapses, then cross tab at the end
# Peel off the index of the pre_to_mn list, which has info on premotor type and segIDs
idx_df = pre_to_mn_df.index.to_frame()
idx_df.index=idx_df.segID
print(idx_df.shape)
idx_df.head()

cnt = 0
xi = 0
delta = 16
xf = delta

pre_to_pre_syn_df = None

timestamp = connectome_create.get_timestamp()

# Iterate over the premotor neurons in chunks
pmn_sids = idx_df.segID.to_list()

while xf<=idx_df.shape[0]:
    sids = pmn_sids[xi:xf]
    pmn_syn_df = client.materialize.synapse_query(pre_ids = sids,timestamp=timestamp) # Takes list
    
    print(pmn_syn_df.shape)

    if pre_to_pre_syn_df is None:
        pre_to_pre_syn_df = pmn_syn_df
    else:
        pre_to_pre_syn_df = pd.concat([pre_to_pre_syn_df,pmn_syn_df],axis=0,join='outer')
    
    cnt=cnt+1
    if cnt % 4 == 0:
        print('{} of {} rows complete, idx = {}'.format(xf,idx_df.shape[0],sids[-1]))
        print(pre_to_pre_syn_df.shape)
    xi=xf
    if xf+delta < idx_df.shape[0]:
        xf=xf+delta
    elif xf==idx_df.shape[0]:
        break
    else:
        xf = idx_df.shape[0]

# This takes more like 4 minutes.
pre_to_pre_syn_df.shape


## Then perform a crosstabulation of sorts

In [ ]:
# OK, this is a crazy piece of code to calculate the crosstabulation
# Currently, the dataframe is too large to perform crosstab. supposedly, pandas 1.4 solves this issue, but I'm having a hard time updating

# first, for readability, just take the columns of interest
df = pre_to_pre_syn_df.loc[:,['pre_pt_root_id','post_pt_root_id']]

# Then use factorize, which encodes each occurance in a list, in this case of paired root_ids, pre and post
ij,tups = pd.factorize(list(zip(*map(df.get,df))))

# Then create a dictionary out of each occurance, storing the number of factorized codes
result = dict(zip(tups, np.bincount(ij)))

# Finally, turn the result into series, with the tuples of pt_root_ids as the multiindex
tupseries = pd.Series(result)

# Finally, threshold by the connection strength and perfom the unstack operation
thresh = 3 # Note, tried this with 5, but then some of the premotors make no connections
thresholdedtupseries = tupseries[tupseries>=thresh]
prelim_pre_to_pre_df = thresholdedtupseries.unstack(fill_value=0)

prelim_pre_to_pre_df = prelim_pre_to_pre_df.T
prelim_pre_to_pre_df.shape

# takes ~ 15 seconds,

In [ ]:
print(prelim_pre_to_pre_df.shape)
pre_to_pre_df = prelim_pre_to_pre_df
print(pre_to_pre_df.shape)

leftout = idx_df[~idx_df.segID.isin(prelim_pre_to_pre_df.columns)]
leftout # nothing should be left out

In [ ]:
connectome_create.save_pre_to_df(pre_to_pre_df,ext='_no_index')